In [ ]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom

In [ ]:
############### Load the dataset ###################
path = 'HIGGS.csv'
classes = 2
data=pd.read_csv(path, header=None)
X=data.iloc[:,1:]
Y=data.iloc[:,0]
X = MinMaxScaler().fit_transform(X)
Y = LabelEncoder().fit_transform(Y)
Y = to_categorical(Y,classes)
print(X.shape,Y.shape)
X_train= X[:10500000]
X_test= X[10500000:]
Y_train= Y[:10500000]
Y_test= Y[10500000:]
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
x_train,x_test = X_train,X_test
n = X_train.shape[1]
# convert 2-D to 3-D.
for i in range(1,n):
  model = Sequential()
  model.add(Dense(n,name='feature', activation='relu',input_shape=(n,)))
  model.add(Dense(classes,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam')
  model.fit(X_train,Y_train,epochs=1,batch_size=128,verbose=0)
  extract = Model(model.inputs, model.get_layer('feature').output)
  features1 = extract.predict(X_train)
  features2 = extract.predict(X_test)
  x_train = np.concatenate((x_train, features1), axis=1) # concatenate on horizontal axis
  x_test = np.concatenate((x_test, features2), axis=1)
print(x_train.shape,x_test.shape)

In [ ]:
from PIL import Image
xtrain = []
xtest = []
for i in range(len(x_train)):
    img = Image.fromarray(x_train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    xtrain.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
xtrain = np.array(xtrain,dtype='float32')
for i in range(len(x_test)):
    img = Image.fromarray(x_test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    xtest.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
xtest = np.array(xtest,dtype='float32')
print(xtrain.shape,xtest.shape)

In [ ]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(xtrain,Y_train,batch_size=128, epochs=20,verbose=1,validation_data=(xtest,Y_test))

In [ ]:
score = model.evaluate(xtest,Y_test)
acc = model.evaluate(xtrain,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])